In [45]:
!pip install glen
from glen import event_detection
import os
import glob
from lxml import etree as ET
import csv
import json
import random
import itertools
import copy
from urllib.parse import urlparse
import re
import math
import sys

In [46]:
def read_json_array(file_path):
    with open(file_path, 'r') as file:
        json_array = json.load(file)
    return json_array

def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def readCSV(input_file, headers=False):
    with open(input_file, 'r',encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        csv_data = [row for row in reader]
        if headers:
            csv_data.pop(0)
    return csv_data

def createCSV(csv_file,data):
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)

def writeFile(fileName,content):
    mkdir(os.path.dirname(fileName))
    with open(fileName, 'w', encoding='utf-8') as f:
        f.write(content)

def appendFile(fileName,content):
    with open(fileName, 'a', encoding='utf-8') as f:
        f.write(content+"\n")

def readFile(fileName):
    with open(fileName, 'r') as file:
        contents = file.read()
    return contents

def createCSV(csv_file,data):
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)

In [47]:
test_data_array=read_json_array("./data/test_annotated_unprepared.json")
sentence_list = []
empty_sentences=[]
for data in test_data_array:
  if data["sentence"]!="":
    sentence_list.append(data["sentence"])
  else:
    empty_sentences.append(data)

In [48]:
result = event_detection(sentence_list, './ckpts', bs_TI=64, bs_TC=64, bs_TR=4, k=10)

load model from ./ckpts/trigger_identifier.bin ...


Trigger Identificaiton: 100%|██████████| 26/26 [01:04<00:00,  2.48s/it]


load model from ./ckpts/type_ranking.bin ...
Predicting 3445 Events ...


Type Ranking: 100%|██████████| 402/402 [01:33<00:00,  4.32it/s]


load model from ./ckpts/type_classifier.bin ...
Data Processing ...


1606it [00:03, 533.08it/s] 


get 10470 predicting data from 1047 events


Type Classification: 100%|██████████| 164/164 [01:39<00:00,  1.65it/s]


In [49]:
print(result)

[{'sen_id': 0, 'sentence': 'fema soldiers goosestepped through the streets .', 'predicted_mentions': []}, {'sen_id': 1, 'sentence': 'independence would be worse for both scotland and the rest of the uk .', 'predicted_mentions': []}, {'sen_id': 2, 'sentence': 'from the mirror up the truck is pretty bad off , but not as bad as the other guy .', 'predicted_mentions': []}, {'sen_id': 3, 'sentence': "globe columnist mike barnicle -- in the second attack on his employer in as many weeks -- averred that his shadowy countenance was so bad , it looked `` like a face you 'd find on a bottle of miracle elixir that promises to do away with diarrhea in our lifetime . ''", 'predicted_mentions': [{'sentence_offset': [172, 176], 'trigger_words': 'find', 'trigger_confidence': 0.9516257047653198, 'event_type': {'xpo_node': 'DWD_Q37112258', 'name': 'finding', 'description': 'process of locating or identifying something'}, 'event_confidence': 0.7301311492919922}, {'sentence_offset': [212, 220], 'trigger_w

In [50]:
mkdir("output")
writeFile("./output/test_predictions.json",str(result))

s_count=0
f_count=0
test_predictions=[["sen_id","sentence","predicted_events"]]
for sentenceData in result:
  row=[]
  row.append(sentenceData["sen_id"])
  row.append(sentenceData["sentence"])
  events=[]
  for prediction in sentenceData["predicted_mentions"]:
    events.append(prediction["event_type"]["name"])
  if len(events)!=0:
    s_count=s_count+1
  else:
    f_count=f_count+1
  row.append(":".join(events))
  test_predictions.append(row)
createCSV("./output/test_predictions.csv",test_predictions)

print("Total sentences in test dataset: "+str(len(test_data_array)))
print("Total non-empty sentences in test dataset: "+str(len(sentence_list)))
print("Total empty sentences in test dataset: "+str(len(empty_sentences)))
print("")
print("Total non-empty sentences in test dataset: "+str(len(sentence_list)))
print("Total non-empty sentences in test dataset with predictions: "+str(s_count))
print("Total non-empty sentences in test dataset without predictions: "+str(f_count))


Total sentences in test dataset: 10627
Total non-empty sentences in test dataset: 1606
Total empty sentences in test dataset: 9021

Total non-empty sentences in test dataset: 1606
Total non-empty sentences in test dataset with predictions: 710
Total non-empty sentences in test dataset without predictions: 896


In [51]:
test_data_array=read_json_array("./data/train_unprepared.json")
sentence_list = []
empty_sentences=[]
for data in test_data_array:
  if data["sentence"]!="":
    sentence_list.append(data["sentence"])
  else:
    empty_sentences.append(data)

In [52]:
result = event_detection(sentence_list, './ckpts', bs_TI=64, bs_TC=64, bs_TR=4, k=10)

load model from ./ckpts/trigger_identifier.bin ...


Trigger Identificaiton: 100%|██████████| 421/421 [17:43<00:00,  2.53s/it]


load model from ./ckpts/type_ranking.bin ...
Predicting 3445 Events ...


Type Ranking: 100%|██████████| 6729/6729 [25:51<00:00,  4.34it/s]


load model from ./ckpts/type_classifier.bin ...
Data Processing ...


26914it [00:55, 489.21it/s]


get 191730 predicting data from 19173 events


Type Classification: 100%|██████████| 2996/2996 [30:15<00:00,  1.65it/s]


In [53]:
print(result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [54]:
mkdir("output")
writeFile("./output/train_predictions.json",str(result))

s_count=0
f_count=0
test_predictions=[["sen_id","sentence","predicted_events"]]
for sentenceData in result:
  row=[]
  row.append(sentenceData["sen_id"])
  row.append(sentenceData["sentence"])
  events=[]
  for prediction in sentenceData["predicted_mentions"]:
    events.append(prediction["event_type"]["name"])
  if len(events)!=0:
    s_count=s_count+1
  else:
    f_count=f_count+1
  row.append(":".join(events))
  test_predictions.append(row)
createCSV("./output/train_predictions.csv",test_predictions)

print("Total sentences in train dataset: "+str(len(test_data_array)))
print("Total non-empty sentences in train dataset: "+str(len(sentence_list)))
print("Total empty sentences in train dataset: "+str(len(empty_sentences)))
print("")
print("Total non-empty sentences in train dataset: "+str(len(sentence_list)))
print("Total non-empty sentences in train dataset with predictions: "+str(s_count))
print("Total non-empty sentences in train dataset without predictions: "+str(f_count))

Total sentences in train dataset: 187468
Total non-empty sentences in train dataset: 26914
Total empty sentences in train dataset: 160554

Total non-empty sentences in train dataset: 26914
Total non-empty sentences in train dataset with predictions: 12692
Total non-empty sentences in train dataset without predictions: 14222


In [55]:
test_data_array=read_json_array("./data/dev_annotated_unprepared.json")
sentence_list = []
empty_sentences=[]
for data in test_data_array:
  if data["sentence"]!="":
    sentence_list.append(data["sentence"])
  else:
    empty_sentences.append(data)

In [56]:
result = event_detection(sentence_list, './ckpts', bs_TI=64, bs_TC=64, bs_TR=4, k=10)

load model from ./ckpts/trigger_identifier.bin ...


Trigger Identificaiton: 100%|██████████| 22/22 [00:55<00:00,  2.51s/it]


load model from ./ckpts/type_ranking.bin ...
Predicting 3445 Events ...


Type Ranking: 100%|██████████| 350/350 [01:21<00:00,  4.31it/s]


load model from ./ckpts/type_classifier.bin ...
Data Processing ...


1398it [00:02, 519.66it/s]


get 11140 predicting data from 1114 events


Type Classification: 100%|██████████| 175/175 [01:45<00:00,  1.65it/s]


In [57]:
print(result)

[{'sen_id': 0, 'sentence': 'what would cause a red indurated isolated area on the lower leg ?', 'predicted_mentions': []}, {'sen_id': 1, 'sentence': "i 'm bad at singing .", 'predicted_mentions': [{'sentence_offset': [12, 19], 'trigger_words': 'singing', 'trigger_confidence': 0.7907474040985107, 'event_type': {'xpo_node': 'DWD_Q109940', 'name': 'duet', 'description': 'musical composition or arrangement for two performers'}, 'event_confidence': 0.9275317192077637}]}, {'sen_id': 2, 'sentence': 'how long does mayonnaise last ? answers related to sell and use by dates , how to store mayonnaise and how to tell if mayonnaise is bad .', 'predicted_mentions': [{'sentence_offset': [40, 47], 'trigger_words': 'related', 'trigger_confidence': 0.8467295169830322, 'event_type': {'xpo_node': 'DWD_Q736922', 'name': 'intimate_relationship', 'description': 'physical or emotional intimacy between 2 or more humans'}, 'event_confidence': 0.8955202102661133}, {'sentence_offset': [51, 55], 'trigger_words': '

In [58]:
mkdir("output")
writeFile("./output/dev_predictions.json",str(result))

s_count=0
f_count=0
test_predictions=[["sen_id","sentence","predicted_events"]]
for sentenceData in result:
  row=[]
  row.append(sentenceData["sen_id"])
  row.append(sentenceData["sentence"])
  events=[]
  for prediction in sentenceData["predicted_mentions"]:
    events.append(prediction["event_type"]["name"])
  if len(events)!=0:
    s_count=s_count+1
  else:
    f_count=f_count+1
  row.append(":".join(events))
  test_predictions.append(row)
createCSV("./output/dev_predictions.csv",test_predictions)

print("Total sentences in dev dataset: "+str(len(test_data_array)))
print("Total non-empty sentences in dev dataset: "+str(len(sentence_list)))
print("Total empty sentences in dev dataset: "+str(len(empty_sentences)))
print("")
print("Total non-empty sentences in dev dataset: "+str(len(sentence_list)))
print("Total non-empty sentences in dev dataset with predictions: "+str(s_count))
print("Total non-empty sentences in dev dataset without predictions: "+str(f_count))

Total sentences in dev dataset: 10359
Total non-empty sentences in dev dataset: 1398
Total empty sentences in dev dataset: 8961

Total non-empty sentences in dev dataset: 1398
Total non-empty sentences in dev dataset with predictions: 697
Total non-empty sentences in dev dataset without predictions: 701
